# Headless CSV file
This notebook shows how to fit a Fermi diad from a CSV file collected at a wide range of wavenumbers


In [1]:
# Importing numpy, pandas and matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Importing DiadFit after its installed. If its not installed, see paper for installation instructions
import DiadFit as pf

## Specify path to folder with your spectra in

In [2]:
# Add the path with your spectra in
mypath=r'G:\My Drive\Berkeley_NEW\DiadFit_outer\docs\Examples\Diad_Spectra'

## Specify your file type, options:
- 'headless_txt',
-'Witec_ASCII'
- 'Renishaw_txt'
- 'HORIBA_txt'
- 'headless_csv'

In [3]:
filetype='headless_csv'
# Specifying the extension here helps to filter out other file types,
# e.g. png files that may be in the folder
file_ext='.CSV'
# If your instrument appends a nubmer like 01, 02 to each file, True removes it
prefix=False

## This function finds all files in your path with the specified file format

In [4]:
Diad_files=pf.get_diad_files(path=mypath, file_ext=file_ext)
Diad_files

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'G:\\My Drive\\Berkeley_NEW\\DiadFit_outer\\docs\\Examples\\Diad_Spectra'

## Select file here OR specify file name
- Here we select the 1st file in the folder [0], if you wanted to target a specific file, you can do that by uncommenting the line below. 

In [ ]:
filename=Diad_files[0] # or put ='FG04_10_MI1_2.txt'
#filename='WITEC_ASCII_MS14_11_MI1_50X.txt'
print(filename)

## Now plot the spectra to inspect what is going on
- By default, DiadFit adds the positions of some common phases in bubbles 

In [ ]:
pf.plot_diad(path=mypath, filename=filename, filetype=filetype)

## After inspecting the spectra, decide if you want to fit the carbonate peak
- If yes, select fit_carbonate=True, else fit_Carbonate=False
- We can see above there is no carbonate, so we select false

In [ ]:
fit_carbonate=False
# Exclude a range here, e.g. if you have a nasty Cosmic ray
exclude_range=None#[1080, 1088]
# This selects the two background positions
lower_range=[1060, 1065]
upper_range=[1120, 1130]
# This sets the peak center, you can tweak if needed
cent=1090
# Amplitude, may need to tweak
amplitude=1000
# Tweak degree of poylnomial fitting the background
N_poly=1
# Select sigma level, e.g. discard points in background N* sigma outside mean
outlier_sigma=12
# N PEAKS - if you want approx peak positionYs
N_peaks=3

# This actually does the work and fits the carbonate peak
# You can tweak the other parameters such as the distance, prominence in the function
Carb_fit, xx_carb, y_carb, result0=pf.fit_carbonate_peak(path=mypath, 
filename=filename, filetype=filetype,
lower_range=lower_range, upper_range=upper_range, 
cent=cent, exclude_range=exclude_range,
N_poly=N_poly, outlier_sigma=outlier_sigma, 
N_peaks=N_peaks, amplitude=amplitude,
distance = 10, prominence=5, width=6, 
threshold=0.1, fit_carbonate=fit_carbonate)
Carb_fit

## Now lets plot the diads
- As with carbonate, you can exclude 2 segments (e.g. around cosmic rays)
- You then specify how many peaks you want to fit on each diad. 1 means just the strong peak, 2 hotbands, and 3 for Diad2 would also fit the C13 peak if present. 
- The identify_diad_peaks then uses scipy findpeaks to identify the positions of the 2 diads, and any other peaks the user asks for. These are saved, and fed into the later functions for peak fitting. 

In [ ]:
# Exclude a range
exclude_range1=None #[1320, 1340]#None
exclude_range2=None #[1236, 1250]
# Specify the number of peaks
n_peaks_diad1=2
n_peaks_diad2=2 
# 
diad1_peaks, diad2_peaks=pf.identify_diad_peaks(path=mypath, filename=filename,
filetype=filetype, n_peaks_diad1=n_peaks_diad1, n_peaks_diad2=n_peaks_diad2,                                      
exclude_range1=exclude_range1,  exclude_range2=exclude_range2,
height = 400, threshold = 0.5, distance = 5, prominence=10, width=0.5)

## If you have no diads visible...
- Uncomment this line, and it will just save the carbonate peak parameters

In [ ]:
#pf.proceed_to_fit_diads(filename, Carb_fit)

## Fit Diad 1
- This function uses the guess of the peak parameters above. You can choose to fit a gaussian background. Else, use a polynomial and specify background positions
- If plot_figure=True, it saves the figure in its own folder in the path, so you can inspect all the fits at a later date. 

In [ ]:
# If you put a number in brackets, it'll center a gaussian near there. 
peak_pos_gauss=None#(1250)
# Degree of polynomial for the background
N_poly_bck_diad1=1
# LH range to use as baseline for the diad
lower_baseline_diad1=[1220, 1250]
# RH range to use as baseline for the diad
upper_baseline_diad1=[1300, 1350]
## Amplitude may need tweaking if you don't get a good match
amplitude=1000
# Do you want to save the figure?
plot_figure=True
# This function does the work fitting the diad
Diad1_fit, result_diad1, best_fit_diad1, xdat_diad1=pf.fit_diad_1_w_bck(
path=mypath, filename=filename, filetype=filetype,
exclude_range1=exclude_range1, exclude_range2=exclude_range2,
N_poly_bck_diad1=N_poly_bck_diad1, lower_baseline_diad1=lower_baseline_diad1,
upper_baseline_diad1=upper_baseline_diad1,
peak_pos_voigt=diad1_peaks, peak_pos_gauss=peak_pos_gauss, amplitude=amplitude,
plot_figure=plot_figure)
Diad1_fit

## Fit diad 2

In [ ]:
# Gaussian or not, either enter approximate center of Guassian, or None
peak_pos_gauss=None#(1378)
# If need to adjust shape of gaussian, enter fwhm here
gauss_sigma=100
gauss_sigma_min=10
gauss_amp=30000
# Degree of polynomail for the background
N_poly_bck_diad2=2
# LH range to use as baseline for the diad
lower_baseline_diad2=[1300, 1360]
# RH range to use as baseline for the diad
upper_baseline_diad2=[1440, 1470]
# Tweak amplitude if needed
amplitude=10000
# This then takes these inputs and feeds them into the function
Diad2_fit, result_diad2, best_fit_diad2, xdat_diad2 =pf.fit_diad_2_w_bck(path=mypath, filename=filename, filetype=filetype, peak_pos_voigt=diad2_peaks,
exclude_range1=exclude_range1, exclude_range2=exclude_range2, N_poly_bck_diad2=N_poly_bck_diad2, 
lower_baseline_diad2=lower_baseline_diad2, upper_baseline_diad2=upper_baseline_diad2,
peak_pos_gauss=peak_pos_gauss, gauss_sigma=gauss_sigma,gauss_fwhm_min=gauss_sigma_min,
gauss_amp=gauss_amp,                                                                         
amplitude=amplitude)
Diad2_fit

## Combine and copy to clipboard!

In [20]:
out=pf.combine_diad_outputs(filename=filename, prefix=prefix, 
Diad1_fit=Diad1_fit, 
Diad2_fit=Diad2_fit, Carb_fit=Carb_fit)
out

,filename,Splitting,Diad1_Cent,Diad1_Area,Residual_Diad1,Diad2_Cent,Diad2_Area,Residual_Diad2,HB1_Cent,HB1_Area,HB2_Cent,HB2_Area,C13_Cent,C13_Area
0,headless_CSV_09_K14.CSV,102.890027,1285.256419,92750.254528,160.300544,1388.146447,142195.933212,244.644775,1264.979694,8542.288503,1409.428468,11182.294974,NaN,0
